In [9]:
pip install "snowflake-connector-python[pandas]"


     |████████████████████████████████| 24.6MB 7.3MB/s eta 0:00:01
     |████████████████████████████████| 450kB 58.3MB/s eta 0:00:01
     |████████████████████████████████| 71kB 3.4MB/s  eta 0:00:01
     |████████████████████████████████| 61kB 5.7MB/s  eta 0:00:01
     |████████████████████████████████| 163kB 105.5MB/s eta 0:00:01
     |████████████████████████████████| 51kB 4.0MB/s  eta 0:00:01
     |████████████████████████████████| 143kB 106.1MB/s eta 0:00:01
     |████████████████████████████████| 204kB 91.0MB/s eta 0:00:01
     |████████████████████████████████| 4.4MB 91.8MB/s eta 0:00:01
     |████████████████████████████████| 194kB 104.2MB/s eta 0:00:01
     |████████████████████████████████| 2.1MB 93.0MB/s eta 0:00:01
     |████████████████████████████████| 71kB 4.3MB/s s eta 0:00:01
     |████████████████████████████████| 512kB 90.8MB/s eta 0:00:01
     |████████████████████████████████| 112kB 103.1MB/s eta 0:00:01
     |████████████████████████████████| 12.4MB 101.5MB/s eta 

Note: you may need to restart the kernel to use updated packages.


In [1]:
from snowflake.snowpark import Session
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=parser['Credentials']['database'],
                         schema=parser['Credentials']['schema'], 
                         role=parser['Credentials']['role'])

    session = Session.builder.configs(connection_params).create()
    return session

/tmp/pip_packages/snowflake/connector/options.py:103: UserWarning: You have an incompatible version of 'pyarrow' installed (10.0.1), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [2]:
session = get_session()

#### Load the prepared data from snowflake

In [3]:
df = session.sql("SELECT * from {table}".format(table="predictive_maintenance_final"))

In [4]:
df.columns

['FAILURE',
 'METRIC5',
 'METRIC6',
 'DAY',
 'DAY_WEEK',
 'SECTOR',
 'OP_PERIOD',
 'DEV_RECONNECTED',
 'MNW1',
 'DIF_M6',
 'DIF_M5',
 'LOG_M2',
 'LOG_M3',
 'LOG_M4',
 'LOG_M7',
 'LOG_M9']

Prepare dataset for training

In [5]:
df_final = df.to_pandas()

In [6]:
def trainer():
    #add all imports
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.metrics import recall_score, f1_score, roc_auc_score, confusion_matrix,classification_report
    from sklearn.linear_model import SGDClassifier
    from sklearn.pipeline import Pipeline
    
    # prepare data for training
    X = df_final.drop(['FAILURE'], axis = 1)
    y = df_final['FAILURE']
    X_train, X_test, Y_train, Y_test = train_test_split(X,y,random_state = 0, test_size=0.25, shuffle = True)
    X_train.reset_index(inplace = True, drop = True)
    Y_train.reset_index(inplace = True, drop = True)

    X_test.reset_index(inplace = True, drop = True)
    Y_test.reset_index(inplace = True, drop = True)
    
    x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, random_state = 0, test_size=0.25)
    
    pipeline = Pipeline([
        ('OHE', OneHotEncoder(handle_unknown = 'ignore',sparse=False)),
#         ('SCALER', StandardScaler()),
        ('SGD', SGDClassifier(random_state=0))
     ])
    
    pipeline.fit(x_train,y_train)
    y_pred = pipeline.predict(x_val) 
    cf_matrix = confusion_matrix(y_val, y_pred)
    cls_report = classification_report(y_val, y_pred)
    print(cls_report)

In [7]:
trainer()

/tmp/pip_packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     23323
           1       0.00      0.00      0.00        20

    accuracy                           1.00     23343
   macro avg       0.50      0.50      0.50     23343
weighted avg       1.00      1.00      1.00     23343



/tmp/pip_packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/pip_packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/pip_packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
from snowflake.snowpark import Session
import snowflake.snowpark.types as T
import snowflake.snowpark.functions as F
from snowflake.snowpark.types import PandasSeries, PandasDataFrame
from snowflake.snowpark.functions import sproc,col,array_construct

@sproc(name="titanic_test", is_permanent=True, stage_location="@MODELS", replace=True, packages=["snowflake-snowpark-python","scikit-learn"])
def train_model(session: Session, features_table: str,m_name:str) -> dict:
    import os
    import time
    from joblib import dump
    start_time = time.time()
    trainer()
    seconds = time.time() - start_time
    time_taken = str(time.strftime("%H:%M:%S",time.gmtime(seconds)))
    
    currentDateAndTime = datetime.now()
    model_filename = "Predictive_Maintenance_model_"+currentDateAndTime.strftime('%Y%m%d_%H%M%S') + '.joblib'
    
    model_file_path = os.path.join('/tmp',model_filename)
    dump(pipeline, model_file_path)
    
    session.file.put(model_file, "@MODELS" ,overwrite=True)

Failed to execute query [queryID: 01af195a-0503-cace-0072-f3030bf3f176] ls '@MODELS'
002003 (02000): SQL compilation error:
Stage 'REFRACT_SNOWFLAKE_INTEGRATION.SNOWFRACT.MODELS' does not exist or not authorized.


SnowparkSQLException: (1304): 01af195a-0503-cace-0072-f3030bf3f176: 002003 (02000): SQL compilation error:
Stage 'REFRACT_SNOWFLAKE_INTEGRATION.SNOWFRACT.MODELS' does not exist or not authorized.